In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Komoran

In [24]:
year_4_data = pd.read_csv("data/4차년도.csv", encoding="cp949")
year_5_data = pd.read_csv("data/5차년도.csv", encoding="cp949")
year_5_data_2 = pd.read_csv("data/5차년도_2차.csv", encoding="cp949")

In [25]:
print(f"4 : {len(year_4_data)}, 5_1 : {len(year_5_data)}, 5_2 : {len(year_5_data_2)}")

4 : 14606, 5_1 : 10011, 5_2 : 19374


In [26]:
data = pd.concat((year_4_data,year_5_data,year_5_data_2))
print(len(data))

43991


In [28]:
# 기쁨 : 0, 슬픔 : 1, 화남 : 2, 두려움 : 3, 역겨움 : 4, 중립 : 5, 놀라움 : 6
emot_indexing_dic = {'happiness' : 0, 'sadness' : 1, 'angry' : 2, 'fear' : 3, 'disgust' : 4, 'neutral' : 5, 'surprise' : 6}


In [31]:
x = []
y = []
for i in range(len(data)):
    emot = [0]*7
    line = data.iloc[i]
    x.append(line['발화문'])
    for j in range(1,6):
        emot[emot_indexing_dic[line[f'{j}번 감정'].lower()]] += line[f'{j}번 감정세기']
    y.append(emot)